災害の種類（地震、洪水、台風、干ばつ、疫病 など）、災害が起きた国・地域、発生日・終了日、死者数、被災者数（負傷者、家失者、影響を受けた人数）、経済的損失額、災害の規模分類（大規模災害かなど）がまとめられているファイルから
各国の多い災害、各災害が起きた回数、死者数をまとめた。

In [11]:
import pandas as pd
import matplotlib.pyplot as plt

# Load the dataset
emdat = pd.read_csv('emdat.csv')

In [15]:
emdat.head(100)  

,Disaster Subgroup,Disaster Type,Disaster Subtype,ISO,Country,Subregion,Region,Latitude,Longitude,Start Year,Total Deaths
0,Meteorological,Storm,Tropical cyclone,USA,United States of America,Northern America,Americas,NaN,NaN,1900,6000.0
1,Hydrological,Flood,Flood (General),JAM,Jamaica,Latin America and the Caribbean,Americas,NaN,NaN,1900,300.0
2,Biological,Epidemic,Viral disease,JAM,Jamaica,Latin America and the Caribbean,Americas,NaN,NaN,1900,30.0
3,Geophysical,Volcanic activity,Ash fall,JPN,Japan,Eastern Asia,Asia,NaN,NaN,1900,30.0
4,Geophysical,Earthquake,Ground movement,TUR,Türkiye,Western Asia,Asia,40.3,43.1,1900,140.0
...,...,...,...,...,...,...,...,...,...,...,...
95,Hydrological,Flood,Riverine flood,CHN,China,Eastern Asia,Asia,NaN,NaN,1911,100000.0
96,Meteorological,Storm,Tropical cyclone,BGD,Bangladesh,Southern Asia,Asia,NaN,NaN,1911,NaN
97,Meteorological,Storm,Tropical cyclone,TWN,Taiwan (Province of China),Eastern Asia,Asia,NaN,NaN,1911,1000.0
98,Climatological,Wildfire,Forest fire,CAN,Canada,Northern America,Americas,NaN,NaN,1911,73.0


In [13]:
def analyze_dominant_disaster_by_country(file_path, top_n=10):

    try:
           df = pd.read_csv(file_path)
    except FileNotFoundError:
        print(f"エラー: ファイルが見つかりません: {file_path}")
        return
    
    df_clean = df.dropna(subset=['Country', 'Disaster Type'])
    
    disaster_counts = df_clean.groupby(['Country', 'Disaster Type']).size().reset_index(name='Count')

    idx = disaster_counts.groupby('Country')['Count'].idxmax()
    dominant_disasters = disaster_counts.loc[idx]
 
    total_country_events = df_clean['Country'].value_counts().reset_index(name='Total Events')
    total_country_events.columns = ['Country', 'Total Events']

    result_df = pd.merge(dominant_disasters, total_country_events, on='Country')
    result_df = result_df.sort_values(by='Total Events', ascending=False)
    
    print(f"\n 国別 - 最も発生件数が多い災害の種類 (上位 {top_n} か国)")
    print("--------------------------------------------------")
    
    display_cols = ['Country', 'Disaster Type', 'Count', 'Total Events']
    display_df = result_df[display_cols].head(top_n)
    
    display_df.columns = ['国名', '最も多い災害', '当該件数', '全災害合計件数']
    
    print(display_df.to_string(index=False))
    
file_name = 'emdat.csv'
analyze_dominant_disaster_by_country(file_name)


 国別 - 最も発生件数が多い災害の種類 (上位 10 か国)
--------------------------------------------------
                      国名 最も多い災害  当該件数  全災害合計件数
United States of America  Storm   742     1172
                   China  Storm   331     1023
                   India  Flood   327      785
             Philippines  Storm   400      710
               Indonesia  Flood   275      636
                   Japan  Storm   198      391
              Bangladesh  Storm   187      368
                  Mexico  Storm   131      311
                  Brazil  Flood   185      289
                Viet Nam  Storm   139      272


In [17]:
import pandas as pd

df = pd.read_csv("emdat.csv")

# 災害タイプ別の死者数合計
deaths = df.groupby("Disaster Type")["Total Deaths"].sum()

# 災害タイプ別の発生件数
counts = df.groupby("Disaster Type")["Total Deaths"].count()

# 2つを1つの表にまとめる
summary = pd.DataFrame({
    "Total Deaths": deaths,
    "Event Count": counts
}).sort_values("Total Deaths", ascending=False)

print("【災害タイプ別：死者数 ＋ 発生件数】")
print(summary)


【災害タイプ別：死者数 ＋ 発生件数】
                             Total Deaths  Event Count
Disaster Type                                         
Drought                        11734272.0           72
Epidemic                        9623111.0         1279
Flood                           7016630.0         4377
Earthquake                      2409975.0         1215
Storm                           1421623.0         3626
Extreme temperature              306324.0          574
Volcanic activity                 86945.0           97
Mass movement (wet)               69393.0          810
Wildfire                           5349.0          232
Mass movement (dry)                4502.0           44
Fog                                4000.0            1
Glacial lake outburst flood         439.0            4
Animal incident                      12.0            1
Infestation                           0.0            0
Impact                                0.0            0


##
感想

分析の結果、国ごとに発生しやすい災害の種類が大きく異なることが分かった。アメリカや中国では嵐が圧倒的に多く、インドやブラジルでは洪水が主な災害となっている。これは、気候帯の違いや地形、季節風、台風・ハリケーンの通り道などの地域特有の環境が災害発生に深く影響していると思った。また、発生件数が多い国ほど複数の災害リスクを抱えており、災害対策の重要性が高いことを改めて実感した。